In [39]:
# The following taken from 2021 notebook

# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math
import random
from collections import Counter
import statistics
import parse
import operator
from functools import reduce

# aliases from utils
getnums = aoc_utils.getnums

# from norvig's pytudes
cat = ''.join

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
data = get_input(20,2019, raw=True)
data = [line.replace("\n","") for line in data]
data


['                                         M     V       Z   Z         L T     J E     W                                           ',
 '                                         L     M       J   Z         C R     Y E     F                                           ',
 '  #######################################.#####.#######.###.#########.#.#####.#.#####.#########################################  ',
 '  #...#.#.#.#.#.#.#...#.....#...#.....#.....#.#.....#...#...#.....#...#...#.....#.#.................#...#.#...#...#.#.....#.#.#  ',
 '  ###.#.#.#.#.#.#.###.#####.#.#.###.#####.###.#.#.#####.#.#.###.#.###.###.#.#####.###.###.#######.#.#.###.###.#.###.###.#.#.#.#  ',
 '  #.#.#.....#...#.#.#.#...#...#.#.........#.....#.#.#...#.#...#.#.......#.#...#.#.#.#.#.....#.#.#.#.........#.#.....#...#.#...#  ',
 '  #.#.#####.###.#.#.#.#.#####.#.#.#.#.###.###.#.###.###.#.###.#.#.###.###.#.###.#.#.###.#####.#.#############.#.###.###.###.#.#  ',
 '  #.......#.......#.....#...#.#...#.#...#...#.#.....#.....#.

In [50]:
def go():
  WALL = '#'
  EMPTY = '.'
  SKIP = ' '

  raw_grid = {}
  xmax = len(data[0])
  ymax = len(data)
  for y,line in enumerate(data):
    for x,c in enumerate(line):
      raw_grid[(x,y)] = c

  portals = []
  grid = {}
  for y in range(ymax):
    for x in range(xmax):
      c = raw_grid[(x,y)]
      if (x,y) in grid: continue
      elif c in [' ', '-']: continue
      elif c in [WALL,EMPTY]: grid[(x,y)] = c
      else:
        assert c in aoc_utils.ALPHABET
        dirs = [
          [(x,y+1),(x,y+2)], # down
          [(x+1,y),(x+2,y)], #right
          [(x-1,y),(x-2,y)], #left
          [(x,y-1),(x,y-2)], #up
        ]
        for diridx,dir in enumerate(dirs):
          if dir[0] in raw_grid and raw_grid[dir[0]] in aoc_utils.ALPHABET:
            if dir[1] not in raw_grid: continue
            if raw_grid[dir[1]] == ' ': continue
            # print(f"x,y {(x,y)}: {c} dir[0] {dir[0]}: {raw_grid[dir[0]]}. dir[1] {dir[1]}: {dir[1] in raw_grid and raw_grid[dir[1]]}")
            assert (dir[1] in raw_grid and raw_grid[dir[1]] == EMPTY)
            grid[dir[1]] = EMPTY
            label = cat([raw_grid[(x,y)], raw_grid[dir[0]]])
            if diridx in [2,3]:
              label = cat(reversed(label))
            portals.append( (label, dir[1]) )
            raw_grid[(x,y)] = '-' # skip this later
            raw_grid[dir[0]] = '-' # skip this later
            break
  portals_map = defaultdict(lambda: [])
  for label,point in portals:
    portals_map[label].append(point)
  for label,points in portals_map.items():
    if label in ['AA','ZZ']: assert len(points) == 1
    else:
      assert len(points) == 2
  point_portals_map = {}
  point_labels_map = {}
  start = None
  goal = None
  for label,points in portals_map.items():
    if label == 'AA': start = points[0]
    if label == 'ZZ': goal = points[0]
    if len(points) == 2:
      p1,p2 = points
      assert p1 not in point_portals_map
      assert p2 not in point_portals_map
      point_portals_map[p1] = p2
      point_portals_map[p2] = p1
      point_labels_map[p1] = point_labels_map[p2] = label
  for p in point_portals_map:
    assert grid[p] == EMPTY

  def solve():
    cost = 0
    cur = start
    init = (cost,cur)
    nodes = [init]
    costs = {}
    while nodes:
      cost,cur = heapq.heappop(nodes)
      if cur == goal: return cost
      if cur in costs and costs[cur] < cost: continue
      costs[cur] = cost
      ns = [n for n in aoc_utils.neighbors4(cur) if n in grid and grid[n] == EMPTY]
      for n in ns: heapq.heappush( nodes, (cost+1, n) )
      if cur in point_portals_map:
        heapq.heappush( nodes, (cost+1, point_portals_map[cur]) )
  return solve()

assert go() == 636


636

In [51]:
ex = """             Z L X W       C                 
             Z P Q B       K                 
  ###########.#.#.#.#######.###############  
  #...#.......#.#.......#.#.......#.#.#...#  
  ###.#.#.#.#.#.#.#.###.#.#.#######.#.#.###  
  #.#...#.#.#...#.#.#...#...#...#.#.......#  
  #.###.#######.###.###.#.###.###.#.#######  
  #...#.......#.#...#...#.............#...#  
  #.#########.#######.#.#######.#######.###  
  #...#.#    F       R I       Z    #.#.#.#  
  #.###.#    D       E C       H    #.#.#.#  
  #.#...#                           #...#.#  
  #.###.#                           #.###.#  
  #.#....OA                       WB..#.#..ZH
  #.###.#                           #.#.#.#  
CJ......#                           #.....#  
  #######                           #######  
  #.#....CK                         #......IC
  #.###.#                           #.###.#  
  #.....#                           #...#.#  
  ###.###                           #.#.#.#  
XF....#.#                         RF..#.#.#  
  #####.#                           #######  
  #......CJ                       NM..#...#  
  ###.#.#                           #.###.#  
RE....#.#                           #......RF
  ###.###        X   X       L      #.#.#.#  
  #.....#        F   Q       P      #.#.#.#  
  ###.###########.###.#######.#########.###  
  #.....#...#.....#.......#...#.....#.#...#  
  #####.#.###.#######.#######.###.###.#.#.#  
  #.......#.......#.#.#.#.#...#...#...#.#.#  
  #####.###.#####.#.#.#.#.###.###.#.###.###  
  #.......#.....#.#...#...............#...#  
  #############.#.#.###.###################  
               A O F   N                     
               A A D   M                     """.split("\n")

In [52]:
# data = ex
def go2():
  WALL = '#'
  EMPTY = '.'

  raw_grid = {}
  xmax = len(data[0])
  ymax = len(data)
  for y,line in enumerate(data):
    for x,c in enumerate(line):
      raw_grid[(x,y)] = c

  xmid = xmax // 2
  ymid = ymax // 2
  center = (xmid, ymid)

  portals = []
  grid = {}
  for y in range(ymax):
    for x in range(xmax):
      c = raw_grid[(x,y)]
      if (x,y) in grid: continue
      elif c in [' ', '-']: continue
      elif c in [WALL,EMPTY]: grid[(x,y)] = c
      else:
        assert c in aoc_utils.ALPHABET
        dirs = [
          [(x,y+1),(x,y+2)], # down
          [(x+1,y),(x+2,y)], # right
          [(x-1,y),(x-2,y)], # left
          [(x,y-1),(x,y-2)], # up
        ]
        for diridx,dir in enumerate(dirs):
          if dir[0] in raw_grid and raw_grid[dir[0]] in aoc_utils.ALPHABET:
            if dir[1] not in raw_grid: continue
            if raw_grid[dir[1]] == ' ': continue
            # print(f"x,y {(x,y)}: {c} dir[0] {dir[0]}: {raw_grid[dir[0]]}. dir[1] {dir[1]}: {dir[1] in raw_grid and raw_grid[dir[1]]}")
            assert (dir[1] in raw_grid and raw_grid[dir[1]] == EMPTY)
            grid[dir[1]] = EMPTY
            label = cat([raw_grid[(x,y)], raw_grid[dir[0]]])
            if diridx in [2,3]:
              label = cat(reversed(label))
            portals.append( (label, dir[1]) )
            raw_grid[(x,y)] = '-' # skip this later
            raw_grid[dir[0]] = '-' # skip this later
            break
  portals_map = defaultdict(lambda: [])
  for label,point in portals:
    portals_map[label].append(point)
  for label,points in portals_map.items():
    if label in ['AA','ZZ']: assert len(points) == 1
    else:
      assert len(points) == 2
  point_portals_map = {}
  point_labels_map = {}
  start = None
  goal = None
  for label,points in portals_map.items():
    if label == 'AA': start = points[0]
    if label == 'ZZ': goal = points[0]
    if len(points) == 2:
      p1,p2 = points
      assert p1 not in point_portals_map
      assert p2 not in point_portals_map
      dp1 = aoc_utils.euclid_dist(p1, center)
      dp2 = aoc_utils.euclid_dist(p2, center)
      inner = p1 if dp1 < dp2 else p2
      point_portals_map[p1] = (p2, 1 if p1 == inner else -1)
      point_portals_map[p2] = (p1, -1 if p1 == inner else 1)
      point_labels_map[p1] = point_labels_map[p2] = label
  for p in point_portals_map:
    assert grid[p] == EMPTY

  def solve():
    cost = 0
    lvl = 0
    cur = start
    init = (cost,lvl,cur)
    nodes = [init]
    costs = {}
    while nodes:
      cost,lvl,cur = heapq.heappop(nodes)
      if cur == goal and lvl == 0: return cost
      if (cur,lvl) in costs and costs[(cur,lvl)] < cost: continue
      costs[(cur,lvl)] = cost
      ns = [n for n in aoc_utils.neighbors4(cur) if n in grid and grid[n] == EMPTY]
      for n in ns:
        if lvl != 0 and n in [start, goal]: continue
        if lvl == 0 and n in point_portals_map and point_portals_map[n][1] == -1: continue
        heapq.heappush( nodes, (cost+1, lvl, n) )
      if cur in point_portals_map:
        portal_point,level_change = point_portals_map[cur]
        heapq.heappush( nodes, (cost+1, lvl + level_change, portal_point) )
  return solve()

# takes 54.6 seconds
assert go2() == 7248


7248